### Imports

In [1]:
import sys
sys.path.append('/hpc/vvib872/Code/tune')

In [2]:
from dask.distributed import Client
import json
import os, time
import sys

import glob, os, dask,math,zarr
import dask.array as da
from PIL import Image
from skimage import io

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-tt8bt5vw because the default path (/tmp/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


### Create a cluster on a separate notebook and create a client here

In [3]:
client = Client('tcp://127.0.0.1:33047')

In [8]:
filepath = "/hpc/vvib872/Data/PSF/PSF149/PSF149_405_000.tif"

dataset = Image.open(filepath)
sample = io.imread(filepath, img_num=1)

UnidentifiedImageError: cannot identify image file '/hpc/vvib872/Data/PSF/PSF149/PSF149_405_000.tif'

In [7]:
sample

array([[ 1,  1,  3, ...,  2,  8,  2],
       [ 4,  1,  0, ...,  4,  2,  1],
       [ 1,  0,  3, ...,  0,  1,  0],
       ...,
       [ 4,  7,  6, ...,  3,  2,  1],
       [16,  8,  4, ...,  0,  3,  8],
       [ 4,  7,  9, ...,  3,  8,  5]], dtype=uint16)

In [6]:
import dask
import dask.array as da

z_range = dataset.n_frames
xy_range = (slice(None,None), slice(None,None))

lazy_arrays = [dask.delayed(io.imread)(filepath, img_num=i) for i in range(z_range)]
lazy_arrays = [da.from_delayed(x[xy_range], shape=sample[xy_range].shape, dtype=sample.dtype)
               for x in lazy_arrays]

da_img = da.stack(lazy_arrays)
da_img

,Array,Chunk
Bytes,746.59 MB,2.10 MB
Shape,"(356, 1024, 1024)","(1, 1024, 1024)"
Count,1424 Tasks,356 Chunks
Type,uint16,numpy.ndarray


In [8]:
chunks = (356,256,256)

shape = tuple(map(lambda x,c: (math.ceil(x/c) * c ),da_img.shape,chunks))

store = zarr.NestedDirectoryStore(filepath + ".zarr")
z_out = zarr.create(shape=shape,chunks=chunks, dtype=da_img.dtype, store=store, overwrite=True, fill_value=0)
z_out

<zarr.core.Array (356, 1024, 1024) uint16>

In [9]:
%%time
da.to_zarr(da_img, z_out)
z_out.info

CPU times: user 189 ms, sys: 31 ms, total: 220 ms
Wall time: 3.73 s


Type,zarr.core.Array
Data type,uint16
Shape,"(356, 1024, 1024)"
Chunk shape,"(356, 256, 256)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.NestedDirectoryStore
No. bytes,746586112 (712.0M)
No. bytes stored,367
Storage ratio,2034294.6
